<a href="https://colab.research.google.com/github/Stefano97/AN2DL_Project2/blob/master/Preprocessing_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [46]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
rm "/content/drive/My Drive/startDataset" -R

In [0]:
!git clone https://github.com/Stefano97/AN2DL_Project2

In [0]:
! pip uninstall kaggle

In [0]:
!pip install kaggle

In [0]:
from google.colab import files

files.upload()

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c ann-and-dl-image-segmentation

In [0]:
!unzip ./*.zip

In [0]:
! rm "/content/drive/My Drive/maschere" -R


In [0]:
! cp -R "/content/ann-and-dl-image-segmentation.zip" "/content/drive/My Drive/"

In [0]:
! ls "drive/My Drive/Segmentation_Dataset/training/masks/img"

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [47]:
%tensorflow_version 2.x
import tensorflow as tf

SEED = 1234
tf.random.set_seed(SEED) 

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_img_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.3)
train_mask_data_gen = ImageDataGenerator(validation_split=0.3)

In [55]:
import os

cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')

# Batch size
bs = 4

# img shape
img_h = 256
img_w = 256

num_classes=21

# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks
training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="training")  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="training")
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="validation")  
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="validation")

train_gen = zip(train_img_gen, train_mask_gen)
valid_gen = zip(valid_img_gen, valid_mask_gen)
os.path.join(training_dir, 'images/img')

Found 5353 images belonging to 1 classes.
Found 5353 images belonging to 1 classes.
Found 2294 images belonging to 1 classes.
Found 2294 images belonging to 1 classes.


'/content/Segmentation_Dataset/training/images/img'

In [83]:
import time
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook
matplotlib.use('agg')


fig = plt.figure()
ax = fig.gca()
fig.show()

iterator = iter(train_gen)

for _ in range(1000):
    img, target = next(iterator)
    img = img[0]   # First element
    img = img * 255  # denormalize
    
    target = np.array(target[0, ..., 0])
    # Assign colors (just for visualization)
    target_img = np.zeros([256, 256, 3])
    
    target_img[np.where(target == 0.0)] = [0, 0, 0]
    target_img[np.where(target == 255.0)] = [255, 255, 255]

    plt.imshow(np.uint8(img))
    fig.canvas.draw()
    time.sleep(1)

    plt.imshow(np.uint8(target_img))
    fig.canvas.draw()
    time.sleep(1)
    break
'''
for _ in range(1000):
    img, target = next(iterator)
    img = img[0]   # First element
    img = img * 255  # denormalize
    target = target[0]

    target = np.array(target[0, ..., 0])
    # Assign colors (just for visualization)
    target_img = np.zeros([256, 256, 3])
    
    target_img[np.where(target == 0.0)] = [0, 0, 0]
    target_img[np.where(target == 255.0)] = [255, 255, 255]
    
    
    ax[0].imshow(np.uint8(img))
    ax[1].imshow(np.uint8(target_img))
    

    fig.canvas.draw()
    break
'''

'\nfor _ in range(1000):\n    img, target = next(iterator)\n    img = img[0]   # First element\n    img = img * 255  # denormalize\n    target = target[0]\n\n    target = np.array(target[0, ..., 0])\n    # Assign colors (just for visualization)\n    target_img = np.zeros([256, 256, 3])\n    \n    target_img[np.where(target == 0.0)] = [0, 0, 0]\n    target_img[np.where(target == 255.0)] = [255, 255, 255]\n    \n    \n    ax[0].imshow(np.uint8(img))\n    ax[1].imshow(np.uint8(target_img))\n    \n\n    fig.canvas.draw()\n    break\n'